In [1]:
import pandas as pd
import sklearn
from scipy import misc
import cv2
import numpy as np 

from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 

from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
import gc

import torch
import torchvision

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [2]:
# dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
# dtree_predictions = dtree_model.predict(X_test) 

# svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
# svm_predictions = svm_model_linear.predict(X_test) 
toTensor = torchvision.transforms.ToTensor()

In [3]:
X_train = np.load('b1_xtrain.npy')
X_test = np.load('b1_xtest.npy')
y_train = np.load('b1_ytrain.npy')
y_test = np.load('b1_ytest.npy')
y_train = [[x] for x in y_train]
y_train = np.array(y_train)

In [4]:
y_train.tolist()

In [ ]:
X_train.tolist()

In [60]:
xTrain = torch.utils.data.DataLoader(X_train, batch_size=1,
                                          shuffle=False, num_workers=8)

yTrain =  torch.utils.data.DataLoader(y_train, batch_size=1,
                                          shuffle=False, num_workers=8)

In [63]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [64]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [63]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

In [17]:
print(X_train.shape)

X_train = toTensor(X_test).to(device)
X_train.shape

(8000, 1000000)


torch.Size([1, 2000, 1000000])

In [ ]:
labels = np.array(['0', '1', '2', '3', '4'])

modelName = "knn"
depth = 7

dsp = plot_confusion_matrix(modelName, X_test, y_test, display_labels=labels, normalize='true')

plt.title("B1: Confusion Matrix w/ " + modelName +  ", N: " + str(depth))

print('Percentage correct: ', 100*np.sum(svm_model_linear.predict(X_test) == y_test)/len(y_test))

with open('B1/Confusion_Matrix'+modelName + '_N_'+str(depth) + '.png', 'wb') as dsp:
    plt.savefig(dsp)